In [1]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')
import numpy as np
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from tkinter import *
from tkinter.filedialog import askopenfilename
import segyio
import lasio
from mayavi import mlab

class mclass:
    def __init__(self,  window):
        self.window = window
        #self.box = Entry(window)
        window.geometry("800x600")
        window.configure(bg='gray')
        self.button = Button (window, text="Load well log", command=self.plot, bg='mint cream', width=18, height=1)
        self.button2 = Button (window, text="Load seismic", command=self.plot_seismic, bg='mint cream', width=18, height=1)
        self.button5 = Button (window, text="Load seismic (npy)", command=self.plot_seismic_npy, bg='mint cream', width=18, height=1)
        self.button3 = Button (window, text="Load elastic parameters", command=self.plot_elastic, bg='mint cream', width=18, height=1)
        self.button4 = Button (window, text="Load porosity", command=self.plot_phi, bg='mint cream', width=18, height=1)

        #self.box.pack ()
        self.button.pack(side = LEFT, anchor=NW, pady = 2)
        self.button2.pack(side = LEFT, anchor=NW, pady = 2)
        self.button5.pack(side = LEFT, anchor=NW, pady = 2)
        self.button3.pack(side = LEFT, anchor=NW, pady = 2)
        self.button4.pack(side = LEFT, anchor=NW, pady = 2)

    def plot (self):
        
        filename = askopenfilename()
        well1 = lasio.read(filename)
        well1_df = well1.df()
        #well1_df_drop = well1_df.dropna(how='any',axis=0)
        data1 = well1_df
        cali = data1['BRCALI']
        res = data1['BRRESP']
        rho = data1['BRDENS']
        dtp = data1['BRDTP']
        #dts = data1['BRDTS']
        phi = data1['BRNEUT']
        gr = data1['BRGR']
        #z = data1['DEPT'][+100:]
        z = data1.index
   

        vp = (10**3/dtp)*0.3048 #km/s
        plt.figure(figsize=(13,20))

        
        plt.subplot(1,5,1)
        #plt.tight_layout()
        plt.plot(cali,z,'gray')
        plt.grid()
        plt.ylim([z[0], z[-1]])
        plt.gca().invert_yaxis()
        plt.xlabel('Caliper (in)', fontsize=15)
        plt.ylabel('Depth (m)', fontsize=16)
        
        plt.subplot(1,5,2)
        #plt.tight_layout()
        plt.plot(rho,z,'k')
        plt.grid()
        plt.ylim([z[0], z[-1]])
        plt.gca().invert_yaxis()
        plt.xlabel('Density (g/cm³)', fontsize=15)

        plt.subplot(1,5,3)
        #plt.tight_layout()
        plt.plot(phi,z,'r')
        plt.grid()
        plt.ylim([z[0], z[-1]])
        plt.gca().invert_yaxis()
        plt.xlabel('Neutron Porosity (%)', fontsize=15)

        plt.subplot(1,5,4)
        #plt.tight_layout()
        plt.plot(vp,z,'b')
        plt.grid()
        plt.ylim([z[0], z[-1]])
        plt.gca().invert_yaxis()
        plt.xlabel('P-wave velocity (km/s)', fontsize=15)

        plt.subplot(1,5,5)
        #plt.tight_layout()
        plt.plot(gr,z,'g')
        plt.grid()
        plt.ylim([z[0], z[-1]])
        plt.gca().invert_yaxis()
        plt.xlabel('Gamma-ray (API)', fontsize=15)


        #canvas = FigureCanvasTkAgg(fig, master=self.window)
        #canvas.get_tk_widget().pack()
        #canvas.draw()
        
        
    def plot_seismic (self):
        filename = askopenfilename()
        
        file2 = segyio.open(filename, mode='r', iline=189, xline=193, strict=True, ignore_geometry=False, endian='big')
        data = segyio.tools.cube(file2)
        #data = np.load(filename, mmap_mode='r')
        #plt.figure(figsize=(15,7))
        #fig = plt.imshow(data[:,50,:].T,cmap='gray',aspect='auto')
        
        #canvas = FigureCanvasTkAgg(fig, master=self.window)
        #canvas.get_tk_widget().pack()
        #canvas.draw()
        print(data.shape)
        fig = mlab.figure(figure='seismic', bgcolor=(1, 1, 1), fgcolor=(0, 0, 0))
        scalars = data   # specifying the data array
        mlab.volume_slice(scalars, slice_index=0, plane_orientation='x_axes', figure=fig,colormap='gray')   # crossline slice
        mlab.volume_slice(scalars, slice_index=0,  plane_orientation='y_axes', figure=fig,colormap='gray')   # inline slice
        mlab.volume_slice(scalars, slice_index=0, plane_orientation='z_axes', figure=fig,colormap='gray')   # depth slice
        mlab.show()
        
        
    def plot_seismic_npy (self):
        filename = askopenfilename()
        data = np.load(filename, mmap_mode='r')
        #plt.figure(figsize=(15,7))
        #fig = plt.imshow(data[:,50,:].T,cmap='gray',aspect='auto')
        
        #canvas = FigureCanvasTkAgg(fig, master=self.window)
        #canvas.get_tk_widget().pack()
        #canvas.draw()
        fig = mlab.figure(figure='seismic', bgcolor=(1, 1, 1), fgcolor=(0, 0, 0))
        scalars = data.T   # specifying the data array
        mlab.volume_slice(scalars, slice_index=0, plane_orientation='x_axes', figure=fig,colormap='gray')   # crossline slice
        mlab.volume_slice(scalars, slice_index=0,  plane_orientation='y_axes', figure=fig,colormap='gray')   # inline slice
        mlab.volume_slice(scalars, slice_index=0, plane_orientation='z_axes', figure=fig,colormap='gray')   # depth slice
        mlab.show()
        
        
        
    def plot_elastic (self):
        filename = askopenfilename()
        
        data = np.load(filename, mmap_mode='r')
        data1 = data[:,:,:,1]
        #data1 = data[:,:,+12:-12,1]
        #plt.figure(figsize=(15,7))
        #plt.imshow(data[:,50,:].T,cmap='gray',aspect='auto')
        
        #canvas = FigureCanvasTkAgg(fig, master=self.window)
        #canvas.get_tk_widget().pack()
        #canvas.draw()
        fig = mlab.figure(figure='elastic', bgcolor=(1, 1, 1), fgcolor=(0, 0, 0))
        scalars = data1   # specifying the data array
        mlab.volume_slice(scalars, slice_index=0, plane_orientation='x_axes', figure=fig,colormap='seismic')   # crossline slice
        mlab.volume_slice(scalars, slice_index=0,  plane_orientation='y_axes', figure=fig,colormap='seismic')   # inline slice
        mlab.volume_slice(scalars, slice_index=0, plane_orientation='z_axes', figure=fig,colormap='seismic')   # depth slice
        mlab.show()
        
        
        
    def plot_phi (self):
        filename = askopenfilename()
        
        data = np.load(filename, mmap_mode='r')
        data1 = data[:,:,:].T
        #plt.figure(figsize=(15,7))
        #plt.imshow(data[:,50,:].T,cmap='gray',aspect='auto')
        
        #canvas = FigureCanvasTkAgg(fig, master=self.window)
        #canvas.get_tk_widget().pack()
        #canvas.draw()
        fig = mlab.figure(figure='phi', bgcolor=(1, 1, 1), fgcolor=(0, 0, 0))
        scalars = data1   # specifying the data array
        mlab.volume_slice(scalars, slice_index=0, plane_orientation='x_axes', figure=fig,colormap='seismic')   # crossline slice
        mlab.volume_slice(scalars, slice_index=0,  plane_orientation='y_axes', figure=fig,colormap='seismic')   # inline slice
        mlab.volume_slice(scalars, slice_index=0, plane_orientation='z_axes', figure=fig,colormap='seismic')   # depth slice
        mlab.show()

window= Tk()


start= mclass (window)


window.mainloop()

********************************************************************************
         to build the TVTK classes (9.2). This may cause problems.
         Please rebuild TVTK.
********************************************************************************

